### Generate simple models with complex model's labels

### Import libraries

In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD, adam
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler, CSVLogger
from keras.models import Model, load_model
from scipy.special import softmax
import pandas as pd
import numpy as np
import pickle

### Import cifar10, normalize and create soft targets

In [ ]:
def getDataset():   
    (x_train, y_train_hard), (x_test, y_test_hard) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    #Z score
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    x_train = (x_train - mean) / (std + 1e-7) 
    x_test = (x_test - mean) / (std + 1e-7)

    num_classes = 10
    y_train_hard = np_utils.to_categorical(y_train_hard, num_classes) #One hot encoding
    y_test_hard = np_utils.to_categorical(y_test_hard, num_classes)

    model = load_model('TestModel.h5') #Loads trained complex net
    ModelCut = Model(inputs=model.input,outputs=model.layers[-2].output) #Gets the same network without last layer
    W = model.layers[-1].get_weights()  #Gets last layer weights

    y_test_logits = np.dot(ModelCut.predict(x_test), W[0]) + W[1]
    y_train_logits = np.dot(ModelCut.predict(x_train), W[0]) + W[1]

    return (x_train, y_train_hard, y_train_logits),(x_test, y_test_hard, y_test_logits)

### Define the model

In [ ]:
 def getSimpleModel(T):         
    num_classes = 10

    # Create the model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_constraint=maxnorm(3),
                     input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Lambda(lambda x: x / T))
    model.add(Activation('softmax'))    
    
    return model

### Import data

In [ ]:
(x_train, y_train_hard, y_train_logits), (x_test, y_test_hard, y_test_logits) = getDataset()

### Loop over different tempeartures

In [ ]:
temperatures = [1, 2, 3, 4, 5]

for i, T in enumerate(temperatures):
    
    model = getSimpleModel(T)
    
    lrate = 0.01
    epochs = 25
    batch_size = 64
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    csv_logger = CSVLogger('training.log', separator=',', append=False)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])  

    # Apply temperature to the soft results
    y_train_soft = softmax(y_train_logits / T, axis = 1)  
    
    # Train model. Change y_train_soft to y_train_hard to train "hard" model
    model.fit(x_train, y_train_soft, callbacks=[csv_logger], validation_data=(x_test, y_test_hard), batch_size=batch_size, epochs=epochs)
    
    # Save model
    model.save('SmallModel_t' + str(T) + '.h5')
    
    # Store train history in csv
    log_data = pd.read_csv('training.log', sep=',', engine='python')
                            
    # Save train history
    pickle.dump(log_data, open('t' + str(T) + '_trainLog.pkl', 'wb'))